In [1]:
from utils.chat_utils import text_to_json
from prompts.eval_dataset_1 import EVALUATION_DATASET_1_PROMPT
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
from utils.anthropic_client import ChatClient
from utils.evaluator import Evaluator

api_key = os.getenv('ANTHROPIC_API_KEY')
if api_key is None:
    raise ValueError("ANTHROPIC_API_KEY not found in environment variables")

messages = []
stop_sequences = ["```"]

params = {
    "messages": messages,
    "stop_sequences": stop_sequences,
    "max_tokens": 4096,
}

# Create ChatClient
chat_client = ChatClient(
    api_key=api_key,
    model="claude-3-haiku-20240307",
    params=params
)

# Create Evaluator for running evaluations
evaluator = Evaluator(chat_client=chat_client)

## Generate Dataset (if needed)

In [3]:
GENERATE_DATASET = False
if GENERATE_DATASET:
    prompt = EVALUATION_DATASET_1_PROMPT
    text = "```json"
    dataset = chat_client.generate_dataset(
        prompt=EVALUATION_DATASET_1_PROMPT, 
        text=text, 
        save_path="datasets/generated_dataset.json"
    )
    print(f"Generated dataset with {len(dataset)} test cases")

## Running the Evaluation

In [4]:
# Test with a simple dataset
test_dataset = [
    {"prompt": "Write a hello world function", "expected_response": "A function that prints hello world"},
    {"prompt": "What is 2+2?", "expected_response": "4"},
    {"prompt": "Explain Python lists", "expected_response": "Explanation of Python lists"}
]

# This should now work without the formatting error!
results = evaluator.run_eval(test_dataset)
print(f"Success! Passed {results['passed_tests']}/{results['total_tests']} tests")

Running test case 1/3...
  ✗ Error in test case 1: 'AnthropicClient' object has no attribute 'params'
Running test case 2/3...
  ✗ Error in test case 2: 'AnthropicClient' object has no attribute 'params'
Running test case 3/3...
  ✗ Error in test case 3: 'AnthropicClient' object has no attribute 'params'

Results saved to eval_results.json

Evaluation complete: 0/3 tests passed (0.00%)
Success! Passed 0/3 tests


In [5]:
import json

# Load the dataset
with open("datasets/generated_dataset.json", "r") as f:
    dataset = json.load(f)

# Run evaluation using the Evaluator class
results = evaluator.run_eval(
    test_dataset=dataset,
    save_results=True,
    results_path="eval_results.json"
)

# Display summary
print("\n" + "="*50)
print("EVALUATION SUMMARY")
print("="*50)
print(f"Total Tests: {results['total_tests']}")
print(f"Passed: {results['passed_tests']}")
print(f"Failed: {results['failed_tests']}")
print(f"Pass Rate: {results['pass_rate']:.2%}")

Running test case 1/3...
  ✗ Error in test case 1: 'AnthropicClient' object has no attribute 'params'
Running test case 2/3...
  ✗ Error in test case 2: 'AnthropicClient' object has no attribute 'params'
Running test case 3/3...
  ✗ Error in test case 3: 'AnthropicClient' object has no attribute 'params'

Results saved to eval_results.json

Evaluation complete: 0/3 tests passed (0.00%)

EVALUATION SUMMARY
Total Tests: 3
Passed: 0
Failed: 3
Pass Rate: 0.00%


## Detailed Results Analysis

In [6]:
# Analyze specific failures
if results['failed_tests'] > 0:
    print("\n" + "="*50)
    print("FAILED TEST CASES")
    print("="*50)
    for result in results['results']:
        if not result.get('passed', False):
            test_case = result.get('test_case', {})
            print(f"\nPrompt: {test_case.get('prompt', 'N/A')[:100]}...")
            if 'error' in result:
                print(f"Error: {result['error']}")
            else:
                grading = result.get('grading_results', {})
                if 'code_grader' in grading:
                    print(f"Code Grader: {grading['code_grader'].feedback}")
                if 'model_grader' in grading:
                    print(f"Model Grader: {grading['model_grader'].feedback}")



FAILED TEST CASES

Prompt: Write a Python function that retrieves the list of EC2 instances in a specific AWS region....
Error: 'AnthropicClient' object has no attribute 'params'

Prompt: Create a JSON object that represents an AWS Lambda function configuration, including the function na...
Error: 'AnthropicClient' object has no attribute 'params'

Prompt: Write a regular expression to validate an AWS S3 bucket name....
Error: 'AnthropicClient' object has no attribute 'params'
